<a href="https://colab.research.google.com/github/lizzldizzl/lissi2/blob/main/Willkommen_bei_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neuer Abschnitt

In [1]:
!pip install pandas

# **Data Governance: Prüfungen mit Kölner Phonetik**

Dieses Notebook verwendet die **Kölner Phonetik**, um Zeichenketten anhand ihrer Lautähnlichkeit zu kodieren und zu vergleichen. Die Kölner Phonetik ist ein phonetischer Algorithmus, der entwickelt wurde, um Schreibvarianten oder Tippfehler in Namen und Wörtern zu erkennen und zu vereinheitlichen. Dadurch lässt sich die Zuverlässigkeit von Such- und Abgleichprozessen in Datenbeständen verbessern.

Mit der **Kölner Phonetik** ist es möglich, Begriffe in eine Ziffernfolge umzuwandeln, die den phonetischen Klang abbildet. Wörter, die ähnlich ausgesprochen werden, erhalten denselben oder einen vergleichbaren Zahlencode, auch wenn sie unterschiedlich geschrieben sind. Dies erleichtert die Identifizierung von Dubletten und die Durchführung von Analysen, die auf Lautähnlichkeiten basieren.


In [9]:
import pandas as pd

In [3]:
! pip install -q kaggle
! mkdir ~/.kaggle

In [4]:
# ... Hier ihre JSON Cred als dictionary eingeben
d_json_cred ={"username":"lizzldizzl","key":"7126d6d48a18986c8a8704fbb94e4a44"}

In [5]:
import pandas as pd
kaggle_cred = pd.DataFrame(d_json_cred, index=[0]).to_json("~/.kaggle/kaggle.json")

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [14]:
!kaggle datasets download -d yagunnersya/fifa-21-messy-raw-dataset-for-cleaning-exploring

Dataset URL: https://www.kaggle.com/datasets/yagunnersya/fifa-21-messy-raw-dataset-for-cleaning-exploring
License(s): CC0-1.0
fifa-21-messy-raw-dataset-for-cleaning-exploring.zip: Skipping, found more recently modified local copy (use --force to force download)


In [15]:
!unzip fifa-21-messy-raw-dataset-for-cleaning-exploring.zip -d ./data

Archive:  fifa-21-messy-raw-dataset-for-cleaning-exploring.zip
replace ./data/fifa21 raw data v2.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ./data/fifa21_raw_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no


In [18]:
CSV_PATH = "./data/fifa21 raw data v2.csv"

# Einlesen des Datensatzes
df = pd.read_csv(CSV_PATH)

# Spalten definieren
NAME_COL = "LongName"        # Spalte mit Spielernamen
ADDRESS_COL = "Nationality"  # Spalte mit Nationalität / Land

print(df[[NAME_COL, ADDRESS_COL]].head())


                       LongName Nationality
0                  Lionel Messi   Argentina
1  C. Ronaldo dos Santos Aveiro    Portugal
2                     Jan Oblak    Slovenia
3               Kevin De Bruyne     Belgium
4    Neymar da Silva Santos Jr.      Brazil


/tmp/ipython-input-3843368696.py:4: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(CSV_PATH)


In [19]:
# %%
import pandas as pd

def koelner_phonetik(word: str) -> str:
    """Wandelt ein deutsches Wort in den Kölner-Phonetik-Code um."""
    word = str(word).upper()
    mapping = {
        "A": 0, "E": 0, "I": 0, "J": 0, "O": 0, "U": 0, "Y": 0,
        "H": None,
        "B": 1, "P": 1,
        "D": 2, "T": 2,
        "F": 3, "V": 3, "W": 3,
        "G": 4, "K": 4, "Q": 4,
        "C": None,  # Spezialfall
        "X": None,  # Spezialfall
        "L": 5,
        "M": 6, "N": 6,
        "R": 7,
        "S": 8, "Z": 8
    }

    def handle_special_cases(w):
        w = w.replace("X", "KS")
        w = w.replace("PH", "F")
        return w

    word = handle_special_cases(word)

    result = []
    for i, char in enumerate(word):
        if char == "C":
            if i == 0:
                if word[i+1:i+2] in "AHKOQUX":
                    result.append(4)
                else:
                    result.append(8)
            else:
                if word[i-1:i] in "SZ":
                    result.append(8)
                elif word[i+1:i+2] in "AHKOQUX":
                    result.append(4)
                else:
                    result.append(8)
        elif char in mapping and mapping[char] is not None:
            result.append(mapping[char])
        elif char in mapping and mapping[char] is None:
            result.append(None)

    cleaned = []
    prev = None
    for num in result:
        if num != prev and num != 0:
            cleaned.append(num)
        prev = num

    return "".join(map(str, cleaned))

In [21]:
def find_similar_but_not_identical(df, name_col, address_col):
    df["name_code"] = df[name_col].astype(str).apply(koelner_phonetik)
    df["address_code"] = df[address_col].astype(str).apply(koelner_phonetik)

    matches = []
    seen_pairs = set()  # ← HINZUGEFÜGT
    grouped = df.groupby(["name_code", "address_code"])

    for _, group in grouped:
        if len(group) > 1:
            for i in range(len(group)):
                for j in range(i+1, len(group)):
                    name1, addr1 = group.iloc[i][name_col], group.iloc[i][address_col]
                    name2, addr2 = group.iloc[j][name_col], group.iloc[j][address_col]

                    # Eindeutiges Paar erzeugen (alphabetisch sortieren)
                    pair = tuple(sorted([(name1, addr1), (name2, addr2)]))

                    if pair in seen_pairs:
                        continue
                    seen_pairs.add(pair)

                    if name1 != name2 or addr1 != addr2:
                        matches.append((name1, name2, addr1, addr2))

    return matches


In [22]:


# Suche nach ähnlichen, aber nicht identischen Einträgen
results = find_similar_but_not_identical(df, NAME_COL, ADDRESS_COL)

# Ausgabe
if results:
    print("Ähnliche, aber nicht identische Einträge gefunden:")
    for n1, n2, a1, a2 in results:
        print(f"- {n1} / {n2}  |  {a1} / {a2}")
    print(f"\nGefundene Paare: {len(results)}")
else:
    print("Keine ähnlichen, aber nicht identischen Einträge gefunden.")


Ähnliche, aber nicht identische Einträge gefunden:
- Boubacar Kamara / Aboubakar Kamara  |  France / France
- Abdulrahman Al Obaid / Abdulrahman Al Obud  |  Saudi Arabia / Saudi Arabia
- Abdullah Al Yousef / Abdullah Al Yousif  |  Saudi Arabia / Saudi Arabia
- Joe Pigott / Joe Piggott  |  England / England
- Paul Coutts / Paul Caddis  |  Scotland / Scotland
- Peng Wang / Yaopeng Wang  |  China PR / China PR
- Peng Wang / Huapeng Wang  |  China PR / China PR
- Yaopeng Wang / Huapeng Wang  |  China PR / China PR
- Ben Garratt / Ben Garrity  |  England / England
- Benjamin Mendy / Benjamin Jeannot  |  France / France
- Ban Suk Oh / Beom Seok Oh  |  Korea Republic / Korea Republic
- Ibrahima Konaté / Brahim Konaté  |  France / France
- Ibrahim Cissé / Ibrahima Cissé  |  France / France
- Ibrahim Sissoko / Ibrahima Sissoko  |  France / France
- Dabao Yu / Taoyu Piao  |  China PR / China PR
- Tobias Karlsson / Tobias Carlsson  |  Sweden / Sweden
- Defu Ma / Da Wen  |  China PR / China PR
- T